# Footfall data exploration

The Leeds city centre footfall data provided by Data Mill North is, quite simply, a mess. While files are consistently provided in a csv format, they have changed several times over the years with column names being added, removed and changed, date formats changing etc.

This notebook explores the data to develop a methodology to clean the data up and provide a single, consistent view of it.

https://datamillnorth.org/dataset/leeds-city-centre-footfall-data

## Objective
The objective of this file is to provide a single, consistent table with the following fields:
* Location
* Timestamp (date and hour)
* Total footfall

This will enable us to build a historic view of footfall figures based on the files that already exist, and build a data pipeline which recognises the data format of any new files so they can be loaded to the DW.

## Retrieve files
Retrieve all files and save them to the files directory, ready for processing.

NOTE: At the time of writing, this step downloads 214 seperate files, totalling 72mb.

In [5]:
import requests
import os
import pandas as pd

local_storage ='files/'

if not os.path.exists(local_storage):
    os.makedirs(local_storage)

r = requests.get('https://datamillnorth.org/api/dataset/leeds-city-centre-footfall-data')
api_response = r.json()

file_list = []
for key, file in api_response['resources'].items():
    if file['format'] == 'csv':
        file_name = file['url'].split('/')[-1]
        url = f'https://datamillnorth.org/download/leeds-city-centre-footfall-data/{key}/{file_name}'

        file_name = file_name.replace('%20',' ')
        response = requests.get(url)
        with open(os.path.join(local_storage, file_name), 'wb') as f:
            f.write(response.content)
        print(f'Downloaded {file_name}')
        file_list.append(file_name)


print('Complete')



Downloaded headrow.csv
Downloaded Monthly Data Feed - Dec 2017.csv
Downloaded briggate-at-mcdonalds.csv
Downloaded Monthy Data Feed - June 2017.csv
Downloaded Copy of Monthly Data Feed-June 2016.csv
Downloaded Monthly Data Feed-April 2017 - 20170510.csv
Downloaded Monthly Data Feed -  Jan 2018.csv
Downloaded Copy of Monthly Data Feed-December 2015 - 20160106.csv
Downloaded monthly-data-feed-oct-2014-20141107.csv
Downloaded Monthly Data Feed-May 2015 - 20150601 (1).csv
Downloaded Monthly Data Feed-Apr 2016 - 20160501.csv
Downloaded Copy of Monthly Data Feed-May 2016 - 20160531.csv
Downloaded Monthly Data Feed - April 2018.csv
Downloaded monthly-data-feed-june-2014-20140710.csv
Downloaded Copy of Monthly Data Feed-March 2017 - 20170403.csv
Downloaded Monthly Data Feed-October 2015 - 20151203.csv
Downloaded albion-street-south.csv
Downloaded Copy of Monthly Data Feed-December 2016 - 20170208.csv
Downloaded dortmund-square.csv
Downloaded Monthly Data Feed-September 2016 - 20161102.csv
Down

Downloaded Weekly Data 04.04.22 - 10.04.22.csv
Downloaded Weekly Data 11.04.22 - 17.04.2022.csv
Downloaded Weekly Data 18.04.22 - 24.04.22.csv
Downloaded Weekly Data 25.04.22 - 01.05.22.csv
Downloaded Weekly Data 02.05.22 - 08.05.22.csv
Downloaded Weekly Data 09.05.22 - 15.05.22.csv
Downloaded Weekly Data 16.05.22 - 22.05.22.csv
Downloaded Weekly Data 23.05.22 - 29.05.22.csv
Downloaded Weekly Data 30.05.22 - 05.06.22.csv
Downloaded Weekly Data 06.06.22 - 12.06.22.csv
Downloaded Weekly Data 13.06.22 - 19.06.22.csv
Downloaded Weekly Data 20.06.22 - 26.06.22.csv
Downloaded Weekly Data 27.06.22 - 03.07.22.csv
Downloaded Weekly Data 04.07.22 - 10.07.22.csv
Downloaded Weekly Data 11.07.22 - 17.07.22.csv
Downloaded Weekly Data 18.07.22 - 24.07.22.csv
Downloaded Weekly Data 25.07.22 - 31.07.22.csv
Downloaded Weekly Data 01.08.22 - 07.08.22.csv
Downloaded Weekly Data 08.08.22 - 14.08.22.csv
Downloaded Weekly Data 15.08.22 - 21.08.22.csv
Downloaded Weekly Data 22.08.22 - 28.08.22.csv
Downloaded 

SyntaxError: 'return' outside function (<ipython-input-5-0c53b35dd61c>, line 25)

## Load to dataframes
Load each of the files into a pandas dataframe for analysis.

At the same time, create a dataframe containing a summary of each file, including the number of rows and the columns in the file, to give us a good idea of each of the different formats we're dealing with.

In [73]:
raw_dataframes = {}
summaries = []
for filename in file_list:
    df_file = pd.read_csv(f'{local_storage}{filename}')
    
    raw_dataframes[filename] = df_file
    
    summary = {
        'filename': filename,
        'columns': sorted(list(df_file.columns)),
        'columns_str': ', '.join(sorted(list(df_file.columns))),
        'columns_cnt': len(list(df_file.columns)),
        'rows': len(df_file)
    }
    summaries.append(summary)

    
df_summary = pd.DataFrame(summaries)
print('Loaded')
    

Loaded


## Examine dataframe types
Let's have a look at the various data formats available, based on the column headers. We'll assign an identifier, column_group, to each unique combination of columns.

In [74]:
# Create a unique identifier for each columns combination
df_summary['column_group'] = df_summary.groupby(['columns_str']).grouper.group_info[0]

# summarise each group
df_columns = (df_summary.groupby(['columns_str', 'column_group'])['rows']
    .agg(['count', 'sum'])
    .reset_index()
    .rename(columns={'count': 'total_files','sum': 'total_rows'}))
    
pd.set_option('display.max_colwidth', None)
df_columns

,columns_str,column_group,total_files,total_rows
0,"2017, 2018, 2019, Unnamed: 1, Unnamed: 10, Unnamed: 11, Unnamed: 12, Unnamed: 2, Unnamed: 3, Unnamed: 5, Unnamed: 6, Unnamed: 7, Unnamed: 9",0,1,79
1,"BRCMonthName, BRCQuarter, BRCWeek, BRCYear, BusinessInCount, BusinessOutCount, BusinessTotalCount, Date, FactoredInCount, FactoredOutCount, FactoredTotalCount, Hour, InCount, LocationGroup, LocationName, OutCount, Sitename, TotalCount, Weekday",1,27,155904
2,"BRCMonthName, BRCQuarter, BRCWeek, BRCYear, Date, Hour, InCount, LocationGroup, LocationName, OutCount, TotalCount, Weekday",2,4,24192
3,"BRCMonthName, BRCQuarter, BRCWeek, BRCYear, Date, Hour, InCount, LocationName, Weekday",3,1,23424
4,"BRCMonthName, BRCWeek, BRCYear, Date, DayOfWeek, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, Location, OutCount, ReportCount, Site",4,3,17280
5,"BRCMonthName, BRCWeek, BRCYear, Date, DayOfWeek, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, Location, OutCount, ReportCount, Site, Unnamed: 14, Unnamed: 15, Unnamed: 16",5,1,6912
6,"BRCMonthName, BRCWeekNum, BRCYear, Date, DayName, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, LocationGroup, LocationName, OutCount, ReportCount, Site",6,152,261744
7,"BRCMonthName, BRCWeekNum, BRCYear, Date, DayName, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, LocationName, OutCount, ReportCount, Site",7,17,60246
8,"Count, Date, Hour, LocationName, Month, WeekDay, WeekNum, Year",8,8,386112


From this summary we can take a few notes:
1. Several files have columns that are coming through as "Unnamed: xx". These can be dropped.
1. There are some consistent fields in each file which we are interested in, such as Date and Hour, and there are others which seem to change name, such as LocationName/Location. Due to the similarities in field names, we can take educated guesses as to which fields are the same.
1. Column group 5 appears to have the same columns as group 6, but has additional Unnamed fields at the end. Once unnamed fields are dropped, this will reduce the group counts down.
1. The number of files that a given set of fields appear in is no indication of how many rows of data are effected. Group 8 has just 8 files with that combination of fields yet it has more rows than group 6 152 files (386112 vs 261744). This is because several of the files summarise long periods of time, whereas others cover just a single week.

### Group 0 - valid file?
Group 0 appears to be wildly different to the rest. Let's examine that file specifically.

In [75]:
# Get the filename
filename = df_summary[(df_summary['column_group']==0)]['filename'].iloc[0]
print(filename)

# Show us that dataframe
raw_dataframes[filename].head()

Christmas analysis.csv


,2017,Unnamed: 1,Unnamed: 2,Unnamed: 3,2018,Unnamed: 5,Unnamed: 6,Unnamed: 7,2019,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Monday,16-Oct-17,128744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tuesday,17-Oct-17,128842.0,NaN,Tuesday,16-Oct-18,131208.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Wednesday,18-Oct-17,135502.0,NaN,Wednesday,17-Oct-18,134874.0,NaN,Wednesday,16-Oct-19,129199.0,NaN,QUITE BUSY: above average footfall for the period
3,Thursday,19-Oct-17,120944.0,NaN,Thursday,18-Oct-18,134372.0,NaN,Thursday,17-Oct-19,134760.0,NaN,VERY BUSY: +30% above average footfall for the period
4,Friday,20-Oct-17,154793.0,NaN,Friday,19-Oct-18,150036.0,NaN,Friday,18-Oct-19,145461.0,NaN,EXTREMELY BUSY: +50% above average footfall for the period


Looking at this file, Christmas analysis.csv, and it is clear that it is some sort of ad-hoc analysis, which is not relevant nor with useful data for our purposes. This can be confirmed by visiting the DataMillNorth page and viewing the notes for this file. It can be excluded when building the historic picture.
It also indicates the possibility of files that are not useful and would result in errors in a data pipeline. We must build in the option to ignore specific files in the pipeline.

# Reload Dataframes
Based on this initial examination, we can reload the dataframes and fix some of the data issues immediately:
1. Ignore Christmas analysis.csv
1. Remove any columns titled Unnamed: xx.

In [108]:
raw_dataframes = {}
summaries = []
for filename in file_list:
    # Ignore the adhoc analysis file
    if filename == 'Christmas analysis.csv':
        continue
        
    df_file = pd.read_csv(f'{local_storage}{filename}')
    
    # Remove any 
    df_file.drop(df_file.columns[df_file.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
        
    
    raw_dataframes[filename] = df_file
    
    summary = {
        'filename': filename,
        'columns': sorted(list(df_file.columns)),
        'columns_str': ', '.join(sorted(list(df_file.columns))),
        'columns_cnt': len(list(df_file.columns)),
        'rows': len(df_file)
    }
    summaries.append(summary)

    
df_summary = pd.DataFrame(summaries)
print('Loaded')

# Create a unique identifier for each columns combination
df_summary['column_group'] = df_summary.groupby(['columns_str']).grouper.group_info[0]

# summarise each group
df_columns = (df_summary.groupby(['columns_str', 'column_group'])['rows']
    .agg(['count', 'sum'])
    .reset_index()
    .rename(columns={'count': 'total_files','sum': 'total_rows'}))
    
pd.set_option('display.max_colwidth', None)
df_columns

Loaded


,columns_str,column_group,total_files,total_rows
0,"BRCMonthName, BRCQuarter, BRCWeek, BRCYear, BusinessInCount, BusinessOutCount, BusinessTotalCount, Date, FactoredInCount, FactoredOutCount, FactoredTotalCount, Hour, InCount, LocationGroup, LocationName, OutCount, Sitename, TotalCount, Weekday",0,27,155904
1,"BRCMonthName, BRCQuarter, BRCWeek, BRCYear, Date, Hour, InCount, LocationGroup, LocationName, OutCount, TotalCount, Weekday",1,4,24192
2,"BRCMonthName, BRCQuarter, BRCWeek, BRCYear, Date, Hour, InCount, LocationName, Weekday",2,1,23424
3,"BRCMonthName, BRCWeek, BRCYear, Date, DayOfWeek, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, Location, OutCount, ReportCount, Site",3,4,24192
4,"BRCMonthName, BRCWeekNum, BRCYear, Date, DayName, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, LocationGroup, LocationName, OutCount, ReportCount, Site",4,152,261744
5,"BRCMonthName, BRCWeekNum, BRCYear, Date, DayName, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, LocationName, OutCount, ReportCount, Site",5,17,60246
6,"Count, Date, Hour, LocationName, Month, WeekDay, WeekNum, Year",6,8,386112


We have cut the number of data formats down to 7 now, each of them unique.

For each format, we're going to view which fields are the right ones we want to use based on a sample file for that data format, and note any other data format issues we need to clear up.

# Examine individual data formats
First we need to grab a sample file for each data format.

In [109]:
df_samples = df_summary[df_summary.groupby('columns_str')['filename'].rank() == 1].reset_index()
df_samples

,index,filename,columns,columns_str,columns_cnt,rows,column_group
0,25,monthly-data-feed-aug-2014-20140904.csv,"[BRCMonthName, BRCQuarter, BRCWeek, BRCYear, Date, Hour, InCount, LocationGroup, LocationName, OutCount, TotalCount, Weekday]","BRCMonthName, BRCQuarter, BRCWeek, BRCYear, Date, Hour, InCount, LocationGroup, LocationName, OutCount, TotalCount, Weekday",12,5376,1
1,31,Copy of Monthly Data Feed-August 2016 - 20160905 (2).csv,"[BRCMonthName, BRCQuarter, BRCWeek, BRCYear, BusinessInCount, BusinessOutCount, BusinessTotalCount, Date, FactoredInCount, FactoredOutCount, FactoredTotalCount, Hour, InCount, LocationGroup, LocationName, OutCount, Sitename, TotalCount, Weekday]","BRCMonthName, BRCQuarter, BRCWeek, BRCYear, BusinessInCount, BusinessOutCount, BusinessTotalCount, Date, FactoredInCount, FactoredOutCount, FactoredTotalCount, Hour, InCount, LocationGroup, LocationName, OutCount, Sitename, TotalCount, Weekday",19,5952,0
2,45,Copy of Monthly Data Feed - Oct 2017.csv,"[BRCMonthName, BRCWeekNum, BRCYear, Date, DayName, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, LocationName, OutCount, ReportCount, Site]","BRCMonthName, BRCWeekNum, BRCYear, Date, DayName, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, LocationName, OutCount, ReportCount, Site",14,224,5
3,48,footfall.csv,"[BRCMonthName, BRCQuarter, BRCWeek, BRCYear, Date, Hour, InCount, LocationName, Weekday]","BRCMonthName, BRCQuarter, BRCWeek, BRCYear, Date, Hour, InCount, LocationName, Weekday",9,23424,2
4,50,Copy of Monthly Data Feed-February 2017 - 20170301.csv,"[BRCMonthName, BRCWeek, BRCYear, Date, DayOfWeek, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, Location, OutCount, ReportCount, Site]","BRCMonthName, BRCWeek, BRCYear, Date, DayOfWeek, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, Location, OutCount, ReportCount, Site",14,5376,3
5,56,albion-street-north.csv,"[Count, Date, Hour, LocationName, Month, WeekDay, WeekNum, Year]","Count, Date, Hour, LocationName, Month, WeekDay, WeekNum, Year",8,47856,6
6,61,01.10.2018 - 31.10.2018.csv,"[BRCMonthName, BRCWeekNum, BRCYear, Date, DayName, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, LocationGroup, LocationName, OutCount, ReportCount, Site]","BRCMonthName, BRCWeekNum, BRCYear, Date, DayName, FactoredInCount, FactoredOutCount, FactoredReportCount, Hour, InCount, LocationGroup, LocationName, OutCount, ReportCount, Site",15,4464,4


Now let's go through and view the sample files. For each group/file we'll determine which field corresponds to our required output field, and put any additional notes on data formats.

Each of the df.head() commands below was uncommented in turn, manually examined and then notes made on which fields correspond to which. Only one group's command has been uncommented below for brevity.

In [133]:
# Group 0
#raw_dataframes['Copy of Monthly Data Feed-August 2016 - 20160905 (2).csv'].head()

# Group 1
#raw_dataframes['monthly-data-feed-aug-2014-20140904.csv'].head()

# Group 2
#raw_dataframes['footfall.csv'].head()

# Group 3
#raw_dataframes['Copy of Monthly Data Feed-February 2017 - 20170301.csv'].head()

# Group 4
#raw_dataframes['01.10.2018 - 31.10.2018.csv'].head()

# Group 5
raw_dataframes['Copy of Monthly Data Feed - Oct 2017.csv'].head()

# Group 6
#raw_dataframes['albion-street-north.csv'].head()

,Site,LocationName,BRCYear,BRCMonthName,BRCWeekNum,DayName,Date,Hour,InCount,OutCount,ReportCount,FactoredInCount,FactoredOutCount,FactoredReportCount,TotalFootfall,FootfallLocation
0,Leeds,Albion Street North,2017,October,40,Monday,02-Oct-17,NaN,28725,0,28725,28725,0,28725,28725,Albion Street North
1,Leeds,Albion Street North,2017,October,40,Tuesday,03-Oct-17,NaN,28303,0,28303,28303,0,28303,28303,Albion Street North
2,Leeds,Albion Street North,2017,October,40,Wednesday,04-Oct-17,NaN,24663,0,24663,24663,0,24663,24663,Albion Street North
3,Leeds,Albion Street North,2017,October,40,Thursday,05-Oct-17,NaN,33531,0,33531,33531,0,33531,33531,Albion Street North
4,Leeds,Albion Street North,2017,October,40,Friday,06-Oct-17,NaN,38260,0,38260,38260,0,38260,38260,Albion Street North


### Conclusions
The groupings and their field mappings are noted below.

|Group|File|Location|Date|Hour|ReportCount|Notes|
|:-|:-|:-|:-|:-|:-|:-|
|0|Copy of Monthly Data Feed-August 2016 - 20160905 (2).csv|LocationName|Date|Hour|TotalCount||
|1|monthly-data-feed-aug-2014-20140904.csv|LocationName|Date|Hour|TotalCount||
|2|footfall.csv|LocationName|Date|Hour|InCount||
|3|Copy of Monthly Data Feed-February 2017 - 20170301.csv|Location|Date|Hour|ReportCount||
|4|01.10.2018 - 31.10.2018.csv|LocationName|Date|Hour|ReportCount||
|5|Copy of Monthly Data Feed - Oct 2017.csv|LocationName|Date|Hour|ReportCount|!Hour field is null|
|6|albion-street-north.csv|LocationName|Date|Hour|Count||

#### Group 5 - Daily, not hourly
Of important note is group 5. The sample file, Copy of Monthly Data Feed - Oct 2017.csv, appears to be broken down by day, not by hour. This indicates that we should filter out files which do not have valid hour values.

#### Date and time formats
After viewing the different files, it is clear that the date and hour formats are inconsistent. Dates are sometimes given as dd-mmm-yy, other times as dd/mm/yy. Hours are sometimes presented as a single hour number and other times as HH:MM. Functions will need to be written to fix these.

#### Renaming count columns
While LocationName, Date and Hour are consistent across files, the actual column which counts the total footfall is not. Some files have Count, TotalCount or InCount instead. To complicate a simple renaming, some files have both ReportCount and one of the others. Therefore a simple renaming will not work. Renaming Count to ReportCount won't work if there's already a ReportCount column. Instead the renaming must only take place if there isn't a ReportCount already.

# Data Cleaning Method

## Date formats
Pandas natively includes a function for automatically converting date formats: pd.to_datetime(). This can be applied to the Date column as a standard import.

In [116]:
test_data = ['02/03/23', '02-Mar-23','2023-03-02']

for value in test_data:
    print(pd.to_datetime(value, infer_datetime_format=True))

2023-02-03 00:00:00
2023-03-02 00:00:00
2023-03-02 00:00:00


## Hour Formats
This function uses regex to extract the hour value from given text and returns it in HH:MM:SS format, ready for conversion to a timestamp.

In [111]:
import re
def clean_hours(x):
    x = str(x)
    try:
        return re.search(r"^([0-9]{1,2})(?:\.|\:|$)", x).group(1).rjust(2,'0') + ':00:00'
    except:
        return '00:00:00'
    
    
test_data = ['01:00','01','1',1]

for value in test_data:
    #data_type = type(value)
    output = clean_hours(value)
    print(f'Value \'{value}\' is reformatted to \'{output}\'')

Value '01:00' is reformatted to '01:00:00'
Value '01' is reformatted to '01:00:00'
Value '1' is reformatted to '01:00:00'
Value '1' is reformatted to '01:00:00'


## Footfall Total Column Name Conversion
This function has an ordered list of potential columns to use as the footfall figure. As soon as it finds the first matching column name, it creates a TotalFootfall column based on that one and then exits the function.

In [112]:
def get_footfall_column(df):
    prefered_source_columns = [
        'ReportCount',
        'TotalCount',
        'InCount',
        'Count'
    ]
    
    for source in prefered_source_columns:
        if source in list(df.columns):
            df['TotalFootfall'] = df[source]
            print(f'Used {source}')
            return df
        
    print('No source column found')
    return False



test_data = [
    'Copy of Monthly Data Feed-August 2016 - 20160905 (2).csv',
    'monthly-data-feed-aug-2014-20140904.csv',
    'footfall.csv',
    'Copy of Monthly Data Feed-February 2017 - 20170301.csv',
    '01.10.2018 - 31.10.2018.csv',
    'Copy of Monthly Data Feed - Oct 2017.csv',
    'albion-street-north.csv'
]

for value in test_data:
    print(f'Attempting to find a source footfall column for {value}...')
    x = get_footfall_column(raw_dataframes[value])

Attempting to find a source footfall column for Copy of Monthly Data Feed-August 2016 - 20160905 (2).csv...
Used TotalCount
Attempting to find a source footfall column for monthly-data-feed-aug-2014-20140904.csv...
Used TotalCount
Attempting to find a source footfall column for footfall.csv...
Used InCount
Attempting to find a source footfall column for Copy of Monthly Data Feed-February 2017 - 20170301.csv...
Used ReportCount
Attempting to find a source footfall column for 01.10.2018 - 31.10.2018.csv...
Used ReportCount
Attempting to find a source footfall column for Copy of Monthly Data Feed - Oct 2017.csv...
Used ReportCount
Attempting to find a source footfall column for albion-street-north.csv...
Used Count


## Location Column Name Conversion
This function follows the exact same pattern as the footfall column conversion.

In [124]:
def get_location_column(df):
    prefered_source_columns = [
        'LocationName',
        'Location'
    ]
    
    for source in prefered_source_columns:
        if source in list(df.columns):
            df['FootfallLocation'] = df[source]
            print(f'Used {source}')
            return df
        
    print(f'No location column found')
    return False

test_data = [
    'Copy of Monthly Data Feed-August 2016 - 20160905 (2).csv',
    'monthly-data-feed-aug-2014-20140904.csv',
    'footfall.csv',
    'Copy of Monthly Data Feed-February 2017 - 20170301.csv',
    '01.10.2018 - 31.10.2018.csv',
    'Copy of Monthly Data Feed - Oct 2017.csv',
    'albion-street-north.csv'
]

for value in test_data:
    print(f'Attempting to find a source location column for {value}...')
    x = get_location_column(raw_dataframes[value])

Attempting to find a source location column for Copy of Monthly Data Feed-August 2016 - 20160905 (2).csv...
Used LocationName
Attempting to find a source location column for monthly-data-feed-aug-2014-20140904.csv...
Used LocationName
Attempting to find a source location column for footfall.csv...
Used LocationName
Attempting to find a source location column for Copy of Monthly Data Feed-February 2017 - 20170301.csv...
Used Location
Attempting to find a source location column for 01.10.2018 - 31.10.2018.csv...
Used LocationName
Attempting to find a source location column for Copy of Monthly Data Feed - Oct 2017.csv...
Used LocationName
Attempting to find a source location column for albion-street-north.csv...
Used LocationName


# Putting it all together...

In [166]:
def get_footfall_column(df, filename):
    prefered_source_columns = [
        'ReportCount',
        'TotalCount',
        'InCount',
        'Count'
    ]
    
    for source in prefered_source_columns:
        if source in list(df.columns):
            df['TotalFootfall'] = df[source]
            return df
        
    print(f'No footfall column found in {filename}')
    return False


def get_location_column(df, filename):
    prefered_source_columns = [
        'LocationName',
        'Location'
    ]
    
    for source in prefered_source_columns:
        if source in list(df.columns):
            df['FootfallLocation'] = df[source]
            return df
        
    print(f'No location column found in {filename}')
    return False


def clean_hours(x):
    x = str(x)
    return re.search(r"^([0-9]{1,2})(?:\.|\:|$)", x).group(1).rjust(2,'0') + ':00:00'

    
    
dataframes_list = []
dropped_rows = 0
for filename in file_list:
    # Ignore the adhoc analysis file
    if filename == 'Christmas analysis.csv':
        continue
        
    df_file = pd.read_csv(f'{local_storage}{filename}')
    
    # Remove any unnamed columns
    df_file.drop(df_file.columns[df_file.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    
    # Get the location column
    df_file = get_location_column(df_file, filename)
    
    # Get the footfall column
    df_file = get_footfall_column(df_file, filename)
    
    # Drop any rows where hour is NA
    missing_hours = df_file['Hour'].isna().sum()
    if missing_hours > 0:
        print(f'Warning! {missing_hours} hour values missing in {filename}.')
        dropped_rows += missing_hours
        df_file = df_file.dropna(subset=['Hour'])
    
    # Resolve the hour value
    df_file['Hour'] = df_file['Hour'].apply(clean_hours)
    
    # Convert the datetime formats
    df_file['Date'] = pd.to_datetime(df_file['Date'], infer_datetime_format=True)
    df_file['Timestamp'] = df_file['Date'].dt.strftime('%d/%m/%Y') + ' ' + df_file['Hour']
    df_file['Timestamp'] = pd.to_datetime(df_file['Timestamp'], format='%d/%m/%Y %H:%M:%S')
    
    # Drop columns we don't need
    df_file = df_file[['FootfallLocation','Timestamp','Date','Hour','TotalFootfall']]
    
    dataframes_list.append(df_file)

df_output = pd.concat(dataframes_list)

# Check and drop duplicates
total_duplicates = len(df_output) - len(df_output.drop_duplicates())
df_output = df_output.drop_duplicates()

df_output.head()

Warning! 248 hour values missing in Monthly Data Feed - Dec 2017.csv.
Warning! 192 hour values missing in Monthly Data Feed-April 2017 - 20170510.csv.
Warning! 248 hour values missing in Monthly Data Feed -  Jan 2018.csv.


<ipython-input-166-f33031f9e48c>:68: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_file['Date'] = pd.to_datetime(df_file['Date'], infer_datetime_format=True)
<ipython-input-166-f33031f9e48c>:68: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_file['Date'] = pd.to_datetime(df_file['Date'], infer_datetime_format=True)


Warning! 280 hour values missing in Copy of Monthly Data Feed - Sept 2017.csv.
Warning! 1152 hour values missing in Copy of Monthly Data Feed-November 2016 - 20161221.csv.


<ipython-input-166-f33031f9e48c>:68: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_file['Date'] = pd.to_datetime(df_file['Date'], infer_datetime_format=True)
<ipython-input-166-f33031f9e48c>:68: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_file['Date'] = pd.to_datetime(df_file['Date'], infer_datetime_format=True)


Warning! 240 hour values missing in Monthly Data Feed - Nov 2017.csv.
Warning! 248 hour values missing in Monthly Data Feed-March 2018.csv.
Warning! 224 hour values missing in Copy of Monthly Data Feed - Oct 2017.csv.
Warning! 224 hour values missing in Monthly Data Feed-Feb 2018.csv.


<ipython-input-166-f33031f9e48c>:68: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_file['Date'] = pd.to_datetime(df_file['Date'], infer_datetime_format=True)


,FootfallLocation,Timestamp,Date,Hour,TotalFootfall
0,Headrow,2008-08-27 00:00:00,2008-08-27,00:00:00,19.0
1,Headrow,2008-08-27 01:00:00,2008-08-27,01:00:00,15.0
2,Headrow,2008-08-27 02:00:00,2008-08-27,02:00:00,14.0
3,Headrow,2008-08-27 03:00:00,2008-08-27,03:00:00,10.0
4,Headrow,2008-08-27 04:00:00,2008-08-27,04:00:00,2.0


## Checks
The total number of rows plus those dropped for the cleaned dataset above should match our total number of rows

In [168]:
total_uncleaned = df_summary['rows'].sum()
total_cleaned = len(df_output)
print(f'The total rows for the cleaned dataset is: {total_cleaned}')
print(f'The total rows for the cleaned dataset plus dropped rows is: {total_cleaned + dropped_rows + total_duplicates}')
print(f'The total rows for the uncleaned dataset is: {total_uncleaned}')

The total rows for the cleaned dataset is: 926374
The total rows for the cleaned dataset plus dropped rows is: 935814
The total rows for the uncleaned dataset is: 935814


In [169]:
print('The total number of missing values is: ' + str(df_output.isna().sum().sum()))

The total number of missing values is: 0


In [170]:
total_duplicates = len(df_output) - len(df_output.drop_duplicates())
print(f'The total duplicate rows is: {total_duplicates}')

The total duplicate rows is: 0


We're just going to quickly do a sense-check of the output dataframe by hour to make sure the values make sense. For example, if a particular hour seems uncharacteristically high (maybe if 3am were higher than 8am), then this would indicate issues with the dataset.

In [172]:
df_output.groupby('Hour')['TotalFootfall'].mean()

Hour
00:00:00     129.508925
01:00:00      99.185984
02:00:00      84.678965
03:00:00      74.193263
04:00:00      51.113478
05:00:00      53.087274
06:00:00     101.591585
07:00:00     244.334801
08:00:00     544.361027
09:00:00     675.986373
10:00:00    1081.051449
11:00:00    1709.967772
12:00:00    2703.506075
13:00:00    2932.417243
14:00:00    2481.525427
15:00:00    2274.299552
16:00:00    2023.778633
17:00:00    1593.617238
18:00:00     909.046009
19:00:00     526.381839
20:00:00     347.604337
21:00:00     260.530712
22:00:00     244.033136
23:00:00     185.435434
Name: TotalFootfall, dtype: float64

The values here look fine. There is a clear progression of foot traffic over a 24 hour period, building up from 7am onwards and tailing off after 5pm. 